In [25]:
import numpy as np
import pandas as pd
import pynapple as nap
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter, rotate
from spatial_manifolds.data.binning import get_bin_config
from spatial_manifolds.data.loading import load_session
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
mouse = 29
day = 23
session_type = 'VR'
sorter = 'kilosort4'
storage = Path('/Users/harryclark/Downloads/nwb_data/')
alpha = 0.001
n_jobs = 8
n_shuffles = 100
seed = 1

class Args:
    def __init__(self,mouse,day,session_type,sorter,storage,alpha,n_jobs,n_shuffles,seed):
        self.mouse = mouse
        self.day = day
        self.session_type = session_type
        self.sorter = sorter
        self.storage = storage
        self.alpha = alpha
        self.n_jobs = n_jobs
        self.n_shuffles = n_shuffles
        self.seed = seed
args = Args(mouse,day,session_type,sorter,storage,alpha,n_jobs,n_shuffles,seed)

if session_type == 'VR':
    tl = 200
elif session_type == 'MCVR':
    tl = 230
bin_config = get_bin_config(session_type)['P']



In [27]:
# Load session
session, session_path, neurons = load_session(args)
tuning_scores = {
    path.with_suffix('').name: pd.read_parquet(path)
    for path in (session_path / 'tuning_scores').iterdir()
}

##### subset based on brain region #####

In [28]:
print(f'cluster locations include {np.unique(neurons.brain_region)}')
in_brain = neurons[neurons.brain_region != 'root'] # remove out of brain units
in_very_specific_region = neurons[neurons.brain_region == 'ENTm2'] # be specific to a location
in_specific_region = neurons[neurons.brain_region.str.startswith('ENT')] # not too specific

cluster locations include ['ENTm1' 'ENTm2' 'ENTm3' 'root']


##### create tuning curve dictionary #####

In [29]:
print(f'tuning curve features can be found by running code:')
print(f'session')
print(f'and then looking for Tsd object')
n_bins = bin_config['num_bins']
bounds = bin_config['bounds']
sigma = bin_config['smooth_sigma']

tcs = {}
for index in neurons.index:
    this_neuron = neurons[neurons.index == index]
    trial_tcs = []
    for tn in session['trials']['trial_number']:
        trial = session['trials'][session['trials']['trial_number'] == tn]
        tc = nap.compute_1d_tuning_curves(this_neuron, 
                                            session["P"], 
                                            nb_bins=n_bins, 
                                            minmax=[bounds[0], bounds[1]],
                                            ep=session["moving"].intersect(trial))[index]
        tc = np.nan_to_num(tc)
        trial_tcs.append(tc)
    trial_tcs = np.array(trial_tcs)
    trial_tcs_flat = trial_tcs.flatten()
    trial_tcs_flat = gaussian_filter(np.nan_to_num(trial_tcs_flat).astype(np.float64), sigma=sigma)
    trial_tcs = trial_tcs_flat.reshape(trial_tcs.shape)
    tcs[index] = trial_tcs

tuning curve features can be found by running code:
session
and then looking for Tsd object
